In [1]:
!pip install torch-geometric


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#Import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [20]:
!nvcc --version

if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
#Test for new dataset
def testFinal(model, indata, device):
    model.eval()
    title = ''
    E_A = 0
    E_A_Ad = 0
    E_A_Ad2 = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          epsilon = 1e-8
          for i in range(len(out)):
            e = ((out[i].item() - real[i].item() ) / (epsilon + out[i].item()))*100
            errorList.append(e)

          errorListAbs = [abs(x) for x in errorList]

          errorListAbsAd2 = errorListAbs[8:12]

          errorListAbs.sort()
          errorListAbsAd = errorListAbs[:-2]

          E_A_el = sum(errorListAbs)/len(errorListAbs)
          E_A_Ad_el = sum(errorListAbsAd)/len(errorListAbsAd)
          E_A_Ad_el2 = sum(errorListAbsAd2)/len(errorListAbsAd2)

          E_A += E_A_el
          E_A_Ad += E_A_Ad_el
          E_A_Ad2 += E_A_Ad_el2
          
          print([round(x,2) for x in errorList])
          print('Avarage Error: '+ str(round(E_A_el,2))+'%, Adjusted Avarage Error: '+ str(round(E_A_Ad_el,2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),4))+'\n')

          #errorAvg += sum(errorListAbs)/len(errorListAbs)

    print('Avarage Tot. Error: '+ str(round(E_A/len(indata),2))+'%\n')
    print('After removing the two worst errors. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad/len(indata),2))+'%')
    print('Total error for nodes 8, 9, 10, 11. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad2/len(indata),2))+'%')
   
#Test for new dataline

def testData(tester_model, datasetTest,line):
    real = datasetTest[line].y
    
    out = tester_model(datasetTest[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out[i].item() - real[i].item())/ out[i] )*100
        if torch.sign(real[i]) == torch.sign(out[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.3f}'.format(round(real[i].item(),5))
        out_formatted = '{:<10.3f}'.format(round(out[i].item(),5))
        error_formatted = '{:<10.2f}'.format(error)
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

In [6]:
#Traindata

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    #zipped = list(zip(dataX, dataY))
    #np.random.shuffle(zipped)
    #dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset

#Getting data
datasetTrain = get_data("C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\1data\\1DatasetTrain.txt")
datasetTest = get_data("C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\1data\\1DatasetTest.txt")


In [14]:
#Data setup
datasetNew = get_data("C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\1data\\1DatasetNew.txt")
print(len(datasetNew))
print(len(datasetTrain))
print(len(datasetTest))

#Splitting dataset

#train_loader = datasetTrain[:int(len(datasetTrain)*0.75)]
#test_loader = datasetTrain[int(len(datasetTrain)*0.75):]

train_loader = datasetTrain[:4000]
test_loader = datasetTrain[-1000:]

print(len(train_loader))
print(len(test_loader))



100
6571
1001
4000
1000


In [8]:
#Data Checks
""""
# one data object from train_loader:
graph = train_loader[55]
print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)
print(graph.x, graph.y)
"""

'"\n# one data object from train_loader:\ngraph = train_loader[55]\nprint(graph.x.shape)\nprint(graph.edge_index.shape)\nprint(graph.y.shape)\nprint(graph.x, graph.y)\n'

In [15]:
# ArchNN1

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True, l1_lambda=0.001):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16) 
        #self.bn0 = nn.BatchNorm1d(16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        #self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()
        self.l1_lambda = l1_lambda

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x


In [11]:
# ArchNN2

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True, l1_lambda=0.001):
        super(ArchNN2, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16) 
        #self.bn0 = nn.BatchNorm1d(16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        #self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        #self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()
        self.l1_lambda = l1_lambda

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        #self.conv4.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        #x = F.elu(self.conv4(x, edge_index))
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x



In [18]:

import time
import torch
import torch.nn.functional as F

def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)

def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):

    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)

def lossAtan(output, real):
    epsilon = 1e-7
    error = ((real - output) / ((real) + epsilon)) 
    error = torch.clamp(error, max=1)   
    return (torch.mean(torch.atan(error) * 100))



def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))

        loss = F.mse_loss(output, data.y.to(device))
        #loss = F.l1_loss(output, data.y.to(device))
        #loss = lossAtan(output, data.y.to(device))

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            total_loss += F.mse_loss(out, data.y.to(device)).item()

            #F.l1_loss(out, data.y.to(device))
            
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)


In [22]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=8).to(device_name)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

run(model, train_loader, test_loader, num_nodes, 300, optimizer, device_name)
torch.save(model.state_dict(), "2304-data1-3000-e300_mse_state_dict.pt")

Epoch: 1/300, Duration: 206.402s,Train Loss: 3.5658, Test Loss:2.5892
Epoch: 2/300, Duration: 219.182s,Train Loss: 1.9862, Test Loss:0.9584
Epoch: 3/300, Duration: 132.330s,Train Loss: 0.4593, Test Loss:0.1463
Epoch: 4/300, Duration: 133.699s,Train Loss: 0.1431, Test Loss:0.0535
Epoch: 5/300, Duration: 132.645s,Train Loss: 0.0902, Test Loss:0.0273
Epoch: 6/300, Duration: 134.880s,Train Loss: 0.0729, Test Loss:0.0234
Epoch: 7/300, Duration: 132.616s,Train Loss: 0.0636, Test Loss:0.0149
Epoch: 8/300, Duration: 131.919s,Train Loss: 0.0564, Test Loss:0.0120
Epoch: 9/300, Duration: 132.543s,Train Loss: 0.0482, Test Loss:0.0114
Epoch: 10/300, Duration: 132.966s,Train Loss: 0.0493, Test Loss:0.0101
Epoch: 11/300, Duration: 133.812s,Train Loss: 0.0396, Test Loss:0.0085
Epoch: 12/300, Duration: 135.561s,Train Loss: 0.0419, Test Loss:0.0097
Epoch: 13/300, Duration: 150.236s,Train Loss: 0.0357, Test Loss:0.0089
Epoch: 14/300, Duration: 144.218s,Train Loss: 0.0346, Test Loss:0.0067
Epoch: 15/300, 

In [152]:
torch.save(model.state_dict(), "2304-data1-3000-e300_state_dict.pt")

#loading
tester_model = ArchNN(num_features, num_nodes, heads=8) 
tester_model.load_state_dict(torch.load("2204-data1-e300_state_dict.pt")) 

tester_model.eval()

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
#More epochs per favoure?
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]
run(model, train_loader, test_loader, num_nodes, 200, optimizer, device_name)

In [24]:
#Test for new dataset

testFinal(model, datasetTest, device_name)


[0.94, 0.96, 0.81, -1.07, -28.4, 1.55, -0.32, 0.14, 0.15, 0.47, 0.6, 0.8, 1.1, 1.15, 2.8, -12.27, 0.41, 0.4, 0.37, 0.48]
Avarage Error: 2.76%, Adjusted Avarage Error: 0.81%
Mean Square: 0.0004

[0.06, 0.54, -0.16, 0.09, -0.05, -0.24, 0.43, 1.03, 0.21, 0.36, 0.49, 0.87, 2.58, -0.11, 0.56, 0.56, 0.55, 0.06, 1.48, 0.39]
Avarage Error: 0.54%, Adjusted Avarage Error: 0.38%
Mean Square: 0.0001

[0.42, -1.2, 0.19, 2.27, 0.43, -7.35, -3.65, 0.87, 0.2, 1.56, 0.49, -0.77, -1.88, -2.54, -1.57, -6.8, 0.04, -0.01, -0.92, -2.22]
Avarage Error: 1.77%, Adjusted Avarage Error: 1.18%
Mean Square: 0.0001

[-14.77, -0.85, -0.28, 0.08, 1.2, 9.37, -3.78, 0.74, 0.33, 0.69, 1.65, 1.05, 2.17, 3.7, 8.59, 0.72, 0.88, 0.64, -0.55, -6.51]
Avarage Error: 2.93%, Adjusted Avarage Error: 1.91%
Mean Square: 0.0001

[-0.38, -0.68, -0.87, 1.45, 5.53, -4.75, 0.47, 0.89, 0.66, -0.21, -0.17, -0.11, -0.87, -0.38, -3.26, 3.1, 1.0, -0.04, 0.05, -0.3]
Avarage Error: 1.26%, Adjusted Avarage Error: 0.83%
Mean Square: 0.0001

[2.9

In [158]:
#Test for new dataline

testData(model, datasetTest,1)

real      out       %error    Sign      
4.303     4.272     -0.73     True      
-0.844    -0.879    3.93      True      
1.293     1.297     0.26      True      
2.235     2.270     1.51      True      
2.203     2.237     1.48      True      
1.485     1.510     1.69      True      
0.398     0.409     2.90      True      
-0.747    -0.735    -1.58     True      
-1.677    -1.654    -1.38     True      
-2.195    -2.152    -1.98     True      
-2.195    -2.177    -0.84     True      
-1.677    -1.668    -0.54     True      
-0.747    -0.722    -3.47     True      
0.398     0.421     5.52      True      
1.485     1.500     0.98      True      
2.203     2.242     1.72      True      
2.235     2.285     2.18      True      
1.293     1.316     1.76      True      
-0.844    -0.822    -2.69     True      
-4.303    -4.222    -1.91     True      


In [155]:
print(tester_model)

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [25]:
testFinal(model, datasetNew, device_name)

[0.53, -1.07, -0.71, 0.99, 0.14, 0.71, 0.6, 0.94, 0.48, -0.11, -0.58, -0.89, 0.18, 0.54, -0.36, 0.38, 0.17, 1.43, -1.59, 0.08]
Avarage Error: 0.62%, Adjusted Avarage Error: 0.53%
Mean Square: 0.0002

[0.58, -0.99, -0.68, 1.01, 0.19, 0.74, 0.72, 1.06, 0.52, -0.1, -0.54, -0.89, 0.07, 0.48, -0.43, 0.33, 0.1, 1.42, -1.75, 0.03]
Avarage Error: 0.63%, Adjusted Avarage Error: 0.53%
Mean Square: 0.0002

[0.63, -0.94, -0.77, 1.06, 0.3, 0.7, 0.88, 1.09, 0.54, -0.09, -0.5, -0.89, -0.06, 0.39, -0.51, 0.27, 0.02, 1.37, -1.91, -0.04]
Avarage Error: 0.65%, Adjusted Avarage Error: 0.54%
Mean Square: 0.0002

[0.66, -0.92, -0.93, 1.14, 0.45, 0.61, 1.05, 1.1, 0.55, -0.07, -0.48, -0.87, -0.27, 0.39, -0.59, 0.25, -0.05, 1.3, -1.99, -0.12]
Avarage Error: 0.69%, Adjusted Avarage Error: 0.58%
Mean Square: 0.0003

[0.69, -0.9, -1.06, 1.19, 0.54, 0.55, 1.15, 1.12, 0.55, -0.05, -0.44, -0.83, -0.41, 0.48, -0.68, 0.23, -0.11, 1.22, -2.05, -0.2]
Avarage Error: 0.72%, Adjusted Avarage Error: 0.62%
Mean Square: 0.000